<a href="https://colab.research.google.com/github/pa0lai/DeepLearning/blob/main/SimpleRNN_EatWhatToday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境設定

In [1]:
# Install HappyML
import os

if not os.path.isdir("HappyML"):
    os.system("git clone https://github.com/cnchi/HappyML.git")

In [2]:
# Upload Dataset
Dataset_File = "Meals.csv"

if not os.path.isfile(Dataset_File):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" + Dataset_File)

In [3]:
# Customizable Constants
train_size = 0.75

win_size = 2 #RNN特有，往前抓N筆
sample_step = 1 #RNN特有，每隔N筆取一次資料
win_moving = 1  #RNN特有，往後移動N格（通常重複一格）

data_batch = 10

# 資料集前處理

In [4]:
# Load Data
import HappyML.preprocessor as pp
import numpy as np

dataset = pp.dataset(file="Meals.csv")

# Remove the date, keep meal only
dataset = pp.onehot_encoder(dataset, columns=[1]).iloc[:, 1:]

In [5]:
# Keep the One Hot Encoding Mapping
mapper = [s[5:] for s in list(dataset.columns)]

# Convert dataset from DataFrame as NDArray
dataset = dataset.values
train_data, test_data = np.split(dataset, [int(train_size * len(dataset))])

# RNN 資料集準備

In [6]:
# Training Set & Testing Set for RNN
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

train_set = TimeseriesGenerator(
        data=train_data,
        targets=train_data,
        length=win_size,
        sampling_rate=sample_step,
        stride=win_moving,
        batch_size=data_batch)

test_set = TimeseriesGenerator(
        data=test_data,
        targets=test_data,
        length=win_size,
        sampling_rate=sample_step,
        stride=win_moving,
        batch_size=data_batch)

# 模型建造

In [7]:
# Create Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# TO-DO: Add more RNN layers & do the validation set
model = Sequential()
model.add(layers.SimpleRNN(10, input_shape=(win_size, dataset.shape[1])))
model.add(layers.Dense(dataset.shape[1], activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])

# 模型訓練

In [8]:
# Train Model
# Run multiple times of RNN can raise up the accuracy.

# TO-DO: Find the best epochs
model.fit(train_set, epochs=50)

Epoch 1/50
3/3 [==============================] - 5s 16ms/step - loss: 1.3910 - acc: 0.2857
Epoch 2/50
3/3 [==============================] - 0s 12ms/step - loss: 1.3511 - acc: 0.2857
Epoch 3/50
3/3 [==============================] - 0s 12ms/step - loss: 1.3240 - acc: 0.2857
Epoch 4/50
3/3 [==============================] - 0s 12ms/step - loss: 1.3052 - acc: 0.2857
Epoch 5/50
3/3 [==============================] - 0s 13ms/step - loss: 1.2902 - acc: 0.2857
Epoch 6/50
3/3 [==============================] - 0s 13ms/step - loss: 1.2722 - acc: 0.2857
Epoch 7/50
3/3 [==============================] - 0s 13ms/step - loss: 1.2583 - acc: 0.4762
Epoch 8/50
3/3 [==============================] - 0s 13ms/step - loss: 1.2416 - acc: 0.4762
Epoch 9/50
3/3 [==============================] - 0s 12ms/step - loss: 1.2294 - acc: 0.4762
Epoch 10/50
3/3 [==============================] - 0s 13ms/step - loss: 1.2136 - acc: 0.4762
Epoch 11/50
3/3 [==============================] - 0s 20ms/step - loss: 1.1997 

# 模型評估

In [9]:
# Model Evaluation
test_loss, test_acc = model.evaluate(test_set)
print("Loss of Test Set:", test_loss)
print("Accuracy of Test Set:", test_acc)

1/1 [==============================] - 0s 259ms/step - loss: 0.7948 - acc: 0.8333
Loss of Test Set: 0.7947983741760254
Accuracy of Test Set: 0.8333333134651184


# 模型預測

In [10]:
# Model Prediction
Y_pred = np.argmax(model.predict(test_set), axis=-1)
Y_real = np.argmax(test_set[0][1], axis=-1)

# Convert from number to label
print("Prediction:", [mapper[i] for i in Y_pred])
print("Real Value:", [mapper[i] for i in Y_real])

1/1 [==============================] - 0s 181ms/step
Prediction: ['黃金炒飯', '大魯麵', '自助餐', '黃金炒飯', '大魯麵', '煎餃']
Real Value: ['自助餐', '大魯麵', '自助餐', '黃金炒飯', '大魯麵', '煎餃']


# 使用者資料預測

In [12]:
# User input
from tensorflow.keras.utils import to_categorical

meals = []
meals += [mapper.index(input("請問你前天吃什麼："))]
meals += [mapper.index(input("請問你昨天吃什麼："))]

meals = np.reshape(to_categorical(meals, dataset.shape[1]), (1, win_size, dataset.shape[1]))
meal_today = np.argmax(model.predict(meals), axis=-1)
print("我猜你今天會吃：", mapper[meal_today[0]])

請問你前天吃什麼：自助餐
請問你昨天吃什麼：自助餐
1/1 [==============================] - 0s 145ms/step
我猜你今天會吃： 大魯麵
